# Pricing for embeddings, from OpenAI website

SPDX-License-Identifier: 0BSD

In [1]:
from tabulate import tabulate

import fr2ex

In [2]:
prices = fr2ex.tokens.find_embedding_model_prices(displayed_only=False)

In [3]:
table = ((model, rate.numerator) for model, rate in prices.items())

tabulate(
    tabular_data=table,
    headers=['Model', '$ / 1K tokens'],
    tablefmt='html',
    floatfmt='.4f',
)

Model,$ / 1K tokens
Ada v2,0.0001
